# 크롤링 실습

## 크롤링 실습을 위한 준비

In [ ]:
!pip3 install requests
!pip3 install bs4
!pip3 install selenium

## GET 요청하기

In [ ]:
import requests

url = 'https://search.naver.com/search.naver?ie=UTF-8&query=%EC%A0%9C%EC%A3%BC%EB%8F%84&sm=chr_hty'
res = requests.get(url)

print(res) # 응답 객체
res.text # HTML 텍스트

#### HTTP 상태 코드
 - 1xx (정보): 요청을 받았으며 프로세스를 계속한다
 - 2xx (성공): 요청을 성공적으로 받았으며 인식했고 수용하였다
 - 3xx (리다이렉션): 요청 완료를 위해 추가 작업 조치가 필요하다
 - 4xx (클라이언트 오류): 요청의 문법이 잘못되었거나 요청을 처리할 수 없다
 - 5xx (서버 오류): 서버가 명백히 유효한 요청에 대해 충족을 실패했다

[출처: 위키피디아](https://ko.wikipedia.org/wiki/HTTP_%EC%83%81%ED%83%9C_%EC%BD%94%EB%93%9C)

In [ ]:
res.status_code # HTTP 상태 코드

## BeautifulSoup을 이용한 크롤링

In [ ]:
# 임시 html 텍스트
html = '''
<html>
  <head>
    <title>Example HTML</title>
  </head>
  <body>
    <div id='first' class='Example'>
      <h3 title='Good Content Title'>Contents Title</h3>
      <p>Test contents</p>
    </div>
    <div id='second' class='Example'>
      <p>Text1</p>
      <p>Text2</p>
      <p>Text3</p>
    </div>
  </body>
</html>
'''

In [ ]:
from bs4 import BeautifulSoup

# 첫 번째 인자: 파싱할 객체
# 두 번째 인자: 사용할 parser(구문 분석기)
soup = BeautifulSoup(html, 'html.parser')

find(), find_all()

태그 검색   
- find(tag, attributes, recursive, text, keywords)   
- find_all(tag, attributes, recursive, text, limit, keywords)   

get_text()

태그 내 텍스트 반환(부모 태그의 경우 자식 태그의 텍스트를 모두 반환)

In [ ]:
tag = soup.find('h3')
print(tag)
tag.get_text()

In [ ]:
tag = soup.find('p')
print(tag)
tag.get_text()

In [ ]:
# find_all은 list 형태로 값 반환
soup.find_all('p')

In [ ]:
soup.find_all('p')[2].get_text()

In [ ]:
# 개수 제한
soup.find_all('p', limit=2)

In [ ]:
# 여러 태그
soup.find_all({'p', 'h3'})

In [ ]:
soup.find('div')

In [ ]:
soup.find('div').get_text().replace('\n', '')

In [ ]:
# 특정 속성을 가진 태그 검색
soup.find('div', {'id': 'second'})

In [ ]:
# keyword 매개변수 이용
soup.find('div', id='second')

In [ ]:
soup.find('div', id='second', class_='Example')

In [ ]:
# 속성이 여러 개일 경우
attrs = {
    'id': 'second',
    'class': 'Example'
}
soup.find('div', attrs)

네이버 금융 주요뉴스 헤드라인 크롤링 예시

In [ ]:
url = "https://finance.naver.com/"
res = requests.get(url)

# 첫번째 인자를 텍스트로 넣어줘야 함
soup = BeautifulSoup(res.text, 'html.parser') # 문서 구조에 따라 'html.parser', 'lxml', 'xml' 등 다양한 구조 분석 방식을 사용할 수 있음

# 주요뉴스 섹션 검색
strategy = soup.find('div', class_="section_strategy")

# 주요뉴스 헤드라인을 저장할 리스트
strategies = []

# 주요뉴스에 있는 헤드라인 텍스트 정보 추출한 뒤 리스트에 저장
for i in range(len(strategy.find_all('a'))-1):
  strategies.append(strategy.find_all('a')[i].get_text())

# 결과 확인
for headline in strategies:
  print(headline)


## Selenium을 이용한 크롤링 - 이 부분은 로컬 환경에서 코드를 실행해 봅시다.

- 크롤링은 Colab이 아닌 개인 로컬환경에서 진행하는 것이 대다수
- Colab에서 Selenium을 진행할 경우 브라우저 창을 띄울 수 없다 -> 크롤링이 잘 진행되고 있는지 직관적으로 확인하기 어려움
- Colab에서 Selenium 패키지를 설치하여 실행하는 코드를 제대로 작성해도 작동하지 않는 경우 발생 -> 따라서 Selenium을 이용하는 경우 Chrome WebDriver를 본인 PC에 맞게 설치하여 로컬환경에서 크롤링을 진행하도록 합시다!   

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정

chrome_options.add_argument('--no-sandbox')

chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
# webdriver에 옵션 적용 가능
options = webdriver.ChromeOptions()

# 브라우저 창이 안뜨고 실행
options.add_argument('headless')
# 브라우저 사이즈 설정
options.add_argument('window-size=1920x1080')

# 옵션 적용
driver = webdriver.Chrome(path, options=options)

In [ ]:
driver.get('https://www.naver.com/')

# id 속성으로 검색창 찾기
search = driver.find_element(By.ID , 'query')

# 검색어 입력
search.send_keys('제주도')
# Keys.RETURN = Enter 입력
search.send_keys(Keys.RETURN)


# a 태그의 텍스트로 찾기
search = driver.find_element(By.LINK_TEXT,'이미지') # 태그 안의 엘리먼트의 내용이 '이미지'인 엘리먼트를 찾는다.
# 클릭
search.click()

\* 셀레니움 사용 시 브라우저에서 해당 웹 페이지의 요소들을 로드하는 데 시간이 좀 걸리기 때문에, element가 존재하지 않는다는 error가 발생할 수 있다.   
그럴 경우 해당 요소가 전부 준비가 될 때까지 대기해야 한다.

```python
driver.implicitly_wait(time_to_wait=5)
```
-> 찾으려는 element가 로드될 때까지 지정한 시간만큼 대기할 수 있도록 설정   
위 방법은 암시적 대기 방식으로, webdriver에 영구적으로 작용된다.   
time_to_wait=5: 5초까지 기다림

https://workingwithpython.com/selenium-waits/

\* cf. 브라우저 로드가 완성될 때까지 단순히 n초를 기다리는 방식은 time.sleep(n) 코드를 통해 실행가능, 각 PC 환경에 따라 브라우저 로드 속도의 차이가 각각 다르므로 driver.implicitly_wait(5)를 사용하는게 속도 측면에서 더 효율적일 수 있다.

```python
time.sleep(5)
```

In [ ]:
# 현재 열려있는 페이지 url 찾기
url = driver.current_url
url
'''

In [ ]:
# 브라우저 닫기
driver.close()

# Selenium을 활용한 크롤링 예시:네이버 토요일, 일요일 웹툰 정보 크롤링
- 로컬 환경에서 진행

### 과제 url

url => https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1

과제: 무신사 코디맵 페이지(많은 댓글 수 기준 정렬)에 나와있는 60개의 코디맵에 대하여 각각 ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers'] 정보를 가져와서 csv 형태로 만든 다음, 해당 코드(.ipynb)와 csv 파일을 본인 github 과제 repository에 제출

Challenge:
1. 각 코디맵마다 클릭한 뒤 다시 이전페이지로 돌아오는 코드를 반복문에 추가하여 ['codimap_explain', 'codimap_hashtag', 'codimap_imgurl'] 정보도 가져오기
2.  page url을 format 함수를 이용해 수정한 뒤 반복문을 통해 1페이지부터 5페이지까지의 코디맵 정보 가져오기

# Challenge는 Optional Task로 필수적으로 해야하는 것은 아닙니다. 좀 더 심화된 크롤링을 원할경우 시도해보면 좋을 것 같아요!

In [48]:
import requests
import bs4
from bs4 import BeautifulSoup

url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
res = requests.get(url, headers=headers)
print(res)
res.text # HTML 텍스트

<Response [200]>


'<!DOCTYPE html>\n<html lang="ko">\n<head>\n\n<!-- Google Tag Manager -->\n<script>\nvar selfCertEnabled = "" || "Y";\nvar THIS_PAGE_GF = \'A\'; // 전역필터 상태값 caching page reload 필요 체크용\nvar dataLayer = window.dataLayer || [];\nwindow.addEventListener("DOMContentLoaded", function() {\n    \n    (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({\'gtm.start\':\n            new Date().getTime(),event:\'gtm.js\'});var f=d.getElementsByTagName(s)[0],\n        j=d.createElement(s),dl=l!=\'dataLayer\'?\'&l=\'+l:\'\';j.async=true;j.src=\n        \'https://www.googletagmanager.com/gtm.js?id=\'+i+dl;f.parentNode.insertBefore(j,f);\n    })(window,document,\'script\',\'dataLayer\',\'GTM-TXDSFSF\');\n}, false);\n</script>\n<!-- End Google Tag Manager -->\n\n<meta charset="UTF-8">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<link rel="shortcut icon" href="/favicon.ico?20200213" type="image/x-icon">\n<meta name="description" content="전문 스타일리스트가 제안하는 다양한 코디로 매일 반복되는 코디 고민을 덜어보세요. 무신사 스토어의 상품들로 구

In [49]:
# 첫번째 인자를 텍스트로 넣어줘야 함
soup = BeautifulSoup(res.text, 'html.parser') # 문서 구조에 따라 'html.parser', 'lxml', 'xml' 등 다양한 구조 분석 방식을 사용할 수 있음

# 스타일 카테고리
category = soup.find_all('a')
items = soup.find_all(class_='style-list-item')

# 카테고리 헤드라인을 저장할 리스트
categories = []
contents = []
titles = []

for item in items:
    style_list_elements = item.find_all(class_='style-list-information__text')
    style_list_titles = item.find_all(class_='style-list-information__title')
    post_information_elements = item.find_all(class_='post-information__text')

    style_list_texts = [element.text for element in style_list_elements]
    post_information_texts = [element.text for element in post_information_elements]
    style_list_titles = [element.text for element in style_list_titles]

    for text in style_list_texts:
        categories.append(text)

    
    for text in post_information_texts:
        contents.append(text)
    
    for text in style_list_titles:
        titles.append(text)


print(categories)
print(names)
print(titles)

['캐주얼', '캐주얼', '캐주얼', '댄디', '캐주얼', '스트릿', '캐주얼', '댄디', '댄디', '캐주얼', '댄디', '댄디', '포멀', '아메리칸 캐주얼', '캐주얼', '캐주얼', '아메리칸 캐주얼', '캐주얼', '캐주얼', '포멀', '댄디', '캐주얼', '댄디', '포멀', '시크', '캐주얼', '캐주얼', '포멀', '캐주얼', '캐주얼', '캐주얼', '캐주얼', '캐주얼', '포멀', '댄디', '아메리칸 캐주얼', '캐주얼', '댄디', '댄디', '아메리칸 캐주얼', '캐주얼', '캐주얼', '아메리칸 캐주얼', '로맨틱', '포멀', '포멀', '포멀', '캐주얼', '걸리시', '스트릿', '캐주얼', '스트릿', '걸리시', '포멀', '캐주얼', '댄디', '캐주얼', '캐주얼', '캐주얼', '캐주얼']
['21.07.02', '조회 16,091', '댓글 49', '21.02.23', '조회 34,681', '댓글 46', '22.03.22', '조회 12,205', '댓글 38', '21.09.15', '조회 20,191', '댓글 27', '21.02.23', '조회 23,543', '댓글 22', '21.02.03', '조회 35,293', '댓글 21', '21.07.30', '조회 39,588', '댓글 18', '21.12.24', '조회 10,316', '댓글 18', '21.03.24', '조회 19,350', '댓글 16', '21.08.31', '조회 20,219', '댓글 16', '21.07.26', '조회 7,863', '댓글 15', '21.12.24', '조회 9,653', '댓글 14', '21.11.09', '조회 6,315', '댓글 13', '21.11.17', '조회 4,338', '댓글 11', '21.10.04', '조회 19,426', '댓글 11', '21.10.08', '조회 16,785', '댓글 10', '21.12.21', '조회 4,496', '댓글 10', '

In [65]:
views=[]
comments = []
dates = []
for i in contents:
    if '조회' in i:
        views.append(i)
    elif '댓글' in i:
        comments.append(i)
    else:
        dates.append(i)
print(views)
print(comments)
print(dates)

['조회 16,091', '조회 34,681', '조회 12,205', '조회 20,191', '조회 23,544', '조회 35,293', '조회 39,591', '조회 10,316', '조회 19,350', '조회 20,219', '조회 7,864', '조회 9,653', '조회 6,315', '조회 4,338', '조회 19,426', '조회 16,785', '조회 4,496', '조회 14,985', '조회 12,675', '조회 11,339', '조회 10,165', '조회 3,141', '조회 16,271', '조회 3,160', '조회 3,718', '조회 3,138', '조회 3,876', '조회 13,732', '조회 9,420', '조회 7,345', '조회 6,963', '조회 16,156', '조회 20,264', '조회 5,655', '조회 3,516', '조회 8,272', '조회 2,696', '조회 16,910', '조회 8,154', '조회 7,002', '조회 12,148', '조회 10,877', '조회 4,690', '조회 2,298', '조회 3,575', '조회 4,239', '조회 3,987', '조회 5,900', '조회 1,919', '조회 4,287', '조회 10,978', '조회 2,802', '조회 4,545', '조회 10,461', '조회 11,401', '조회 2,570', '조회 5,431', '조회 2,704', '조회 3,440', '조회 11,483']
['댓글 49', '댓글 46', '댓글 38', '댓글 27', '댓글 22', '댓글 21', '댓글 18', '댓글 18', '댓글 16', '댓글 16', '댓글 15', '댓글 14', '댓글 13', '댓글 11', '댓글 11', '댓글 10', '댓글 10', '댓글 10', '댓글 9', '댓글 9', '댓글 8', '댓글 8', '댓글 8', '댓글 8', '댓글 8', '댓글 8', '댓글 8', '댓글 8', '댓글 7', '

In [66]:
import pandas as pd

In [67]:
print(len(categories))
print(len(titles))
print(len(dates))

60
60
60


In [69]:
musinsa = pd.DataFrame({'codimap_category':categories,
                             'codimap_title':titles,
                             'codimap_date':dates,
                       'views':views,
                       'comment_numbers':comments})

In [72]:
musinsa.to_csv('/Users/jaehyunoh/Desktop/YBIGTA/0707 크롤링/musinsa.csv',encoding='cp949')